In [ ]:
import mmf_setup;mmf_setup.nbinit()

Here we consider integration of the BdG equations at $T=0$.  In particular, we identify when the integrands might have kinks in order to specify points of integration.  We start with the quasi-particle dispersion relationships which define the occupation numbers.  We allow for Fulde-Ferrell states with momentum $q$ along the $x$ axis. Kinks occur when these change sign:

$$
  \omega_{\pm} = \epsilon_{\pm} \pm E, \qquad
  E = \sqrt{\epsilon_+^2+\abs{\Delta}^2},\\
  \epsilon_{\pm} = \frac{\epsilon_a \pm \epsilon_b}{2}, \qquad
  \epsilon_{a, b} = \frac{(p_x \pm q)^2 + p_\perp^2}{2m} - \mu_{a,b}.
$$

Simplifying, we have:

$$
  \epsilon_{-} = \frac{qp_x}{m} - \mu_{-}, 
  \epsilon_{+} = \frac{p_x^2 + p_\perp^2}{2m} - \Bigl(\overbrace{\mu_{+} - \frac{q^2}{2m}}^{\mu_q}\Bigr), 
  \qquad
  \mu_{\pm} = \frac{\mu_{a} \pm \mu_{b}}{2}.
$$

Critical points occur when $\omega_{\pm} = 0$ which gives the following conditions:

$$
  \epsilon_-^2 = \epsilon_+^2 + \abs{\Delta}^2\\
  \left(\frac{qp_x}{m} - \mu_{-}\right)^2 = \left(\frac{p_x^2 + p_\perp^2}{2m} - \mu_q\right)^2 + \abs{\Delta}^2.
$$

This may be solved for $p_\perp$ giving the following critical points:

$$
  p_\perp^2 = 2m\left(\mu_q \pm \sqrt{\left(\frac{qp_x}{m} - \mu_{-}\right)^2 - \abs{\Delta}^2}\right) - p_x^2.
$$

In [ ]:
%pylab inline --no-import-all
from ipywidgets import interact
from mmfutils.plot import imcontourf
m = 1

mu = eF = 1.0
pF = np.sqrt(2*m*eF)
delta = 0.5*mu
p_max = 2*pF

@interact(q=(0, 2, 0.1), dmu=(-1.0, 1.0, 0.1), delta=(0, 1, 0.1))
def plot_regions(q=0, dmu=0.4, delta=0.2):
    delta = np.abs(delta)
    p_x, p_perp = np.meshgrid(*(np.linspace(0, p_max, 500),)*2,
                              indexing='ij', sparse=True)
    mu_a = mu + dmu
    mu_b = mu - dmu
    e_a = ((p_x+q)**2+p_perp**2)/2/m - mu_a
    e_b = ((p_x-q)**2 + p_perp**2)/2/m - mu_b
    e_p, e_m = (e_a + e_b)/2.0, (e_a-e_b)/2.0
    E = np.sqrt(e_p**2 + delta**2)
    w_p, w_m = e_m + E, e_m - E
    
    # Analytic regions
    p_x_ = p_x.ravel()
    mu_q = mu - q**2/2/m
    p_perp1_ = np.ma.sqrt(2*m*(mu_q + np.sqrt((q*p_x_/m - dmu)**2 - delta**2)) - p_x_**2
                         ).filled(np.nan)
    p_perp2_ = np.ma.sqrt(2*m*(mu_q - np.sqrt((q*p_x_/m - dmu)**2 - delta**2)) - p_x_**2
                         ).filled(np.nan)
    p_x_special = np.ma.divide(m*(dmu - np.array([delta, -delta])), q).filled(np.nan)
    plt.figure(figsize=(10,5))
    plt.subplot(121)
    imcontourf(p_x, p_perp, np.sign(w_p))
    plt.plot(p_x_, p_perp1_, p_x, p_perp2_)
    plt.vlines(p_x_special, 0, p_max); plt.xlim(0, p_max)
    plt.xlabel('p_x');plt.ylabel('p_perp')
    plt.subplot(122)    
    imcontourf(p_x, p_perp, np.sign(-w_m))
    plt.plot(p_x_, p_perp1_, p_x, p_perp2_)
    plt.vlines(p_x_special, 0, p_max); plt.xlim(0, p_max)
    plt.xlabel('p_x');plt.ylabel('p_perp')

$$
  2p_\perp\d{p_\perp} = 2m\left(\mu_q \pm \sqrt{\left(\frac{qp_x}{m} - \mu_{-}\right)^2 - \abs{\Delta}^2}\right) - p_x^2.
$$



We thus have the correct regions.  The second task is to figure out where these regions start and end so we can indentify special points in $p_x$.  These occur when $\d{p_x}/\d{p_\perp} = 0$, or when the right-hand side of the following expression diverges.  One condition is simple, but the condition that $p_\perp = 0$ requires solving a quartic polynomial:

$$
  \frac{\d{p_\perp}}{\d{p_x}} = \frac{1}{2p_\perp}\left(\pm \frac{2\left(\frac{qp_x}{m} - \mu_{-}\right)q}{\sqrt{\left(\frac{qp_x}{m} - \mu_{-}\right)^2 - \abs{\Delta}^2}} - 2 p_x\right), \\
  \left(\frac{qp_x}{m} - \mu_{-}\right)^2 - \abs{\Delta}^2 = 0,\\
  p_x = \frac{m(\mu_{-} \pm \abs{\Delta})}{q}, \qquad
  \left(\frac{qp_x}{m} - \mu_{-}\right)^2 - \abs{\Delta}^2 = \left(\frac{p_x^2}{2m} - \mu_q\right)^2.
$$
